# Asymmetric Encryption: RSA Algorithm

## Contents
1. RSA Implementation
2. How RSA Works
3. Attack Methods
4. Symmetric vs Asymmetric Comparison

## Prerequisites:

```bash
pip install cryptography pycroptodome
```

In [9]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import time
import math

## 1. RSA Implementation

In [3]:
# Generate RSA keys
private_key = rsa.generate_private_key(
    public_exponent=65537, key_size=2048, backend=default_backend()
)
public_key = private_key.public_key()


def rsa_encrypt(message, public_key):
    return public_key.encrypt(
        message.encode("utf-8"),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
        ),
    )


def rsa_decrypt(ciphertext, private_key):
    return private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
        ),
    ).decode("utf-8")


# Example
message = "Secret message"
encrypted = rsa_encrypt(message, public_key)
decrypted = rsa_decrypt(encrypted, private_key)

print(f"Original: {message}")
print(f"Encrypted: {encrypted[:50].hex()}...")
print(f"Decrypted: {decrypted}")
print(f"Match: {message == decrypted}")

Original: Secret message
Encrypted: 7d9114c3cf6ac99d54cd82cb903efbf85100b10b4618d6102d62db24330f10494224df65bff3e01e0d3a57ac28532fcf9b4e...
Decrypted: Secret message
Match: True


## 2. How RSA Works

**Key Generation:**
1. Choose primes p, q
2. Calculate n = p × q
3. Calculate φ(n) = (p-1)(q-1)
4. Choose e (usually 65537)
5. Calculate d ≡ e⁻¹ (mod φ(n))

**Encryption/Decryption:**
- Encrypt: C = M^e mod n
- Decrypt: M = C^d mod n

**Security:** Based on difficulty of factoring large numbers.

In [ ]:
# Simple RSA example with small numbers
p, q = 61, 53
n = p * q  # 3233
phi = (p - 1) * (q - 1)  # 3120
e = 17
d = pow(e, -1, phi)  # 2753

message = 42
ciphertext = pow(message, e, n)
decrypted = pow(ciphertext, d, n)

print(f"p={p}, q={q}, n={n}")
print(f"Public key: (e={e}, n={n})")
print(f"Private key: (d={d}, n={n})")
print(f"Message: {message} -> Encrypted: {ciphertext} -> Decrypted: {decrypted}")

p=61, q=53, n=3233
Public key: (e=17, n=3233)
Private key: (d=2753, n=3233)
Message: 42 → Encrypted: 2557 → Decrypted: 42


## 3. Attack Methods

### 3.1 Factorization Attack
**Concept:** Factor n to find p and q, then calculate private key.  
**Difficulty:** Infeasible for large keys (2048+ bits).  
**Mitigation:** Use sufficiently large keys.

In [ ]:
def factor(n):
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return i, n // i
    return None, None


# Attack small key
n_small = 3233
p_found, q_found = factor(n_small)
print(f"Factoring n={n_small}: p={p_found}, q={q_found}")
print("Small keys are vulnerable. Real RSA (2048-bit) takes billions of years to factor.")

Factoring n=3233: p=53, q=61
⚠️ Small keys are vulnerable. Real RSA (2048-bit) takes billions of years to factor.


### 3.2 Other Attacks

**Small Exponent Attack:** Exploits small e values. *Mitigation: Use e=65537 with OAEP.*

**Timing Attack:** Measures decryption time to deduce key. *Mitigation: Constant-time algorithms.*

**Chosen Ciphertext Attack:** Manipulates ciphertexts without padding. *Mitigation: Always use OAEP.*

**Quantum Computing:** Shor's algorithm can factor in polynomial time. *Mitigation: Post-quantum cryptography.*

## 4. Symmetric vs Asymmetric Comparison

In [6]:
# AES (Symmetric) Implementation
aes_key = get_random_bytes(32)


def aes_encrypt(message, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct = cipher.encrypt(pad(message.encode("utf-8"), AES.block_size))
    return cipher.iv + ct


def aes_decrypt(ciphertext, key):
    iv, ct = ciphertext[:16], ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct), AES.block_size).decode("utf-8")


# Performance comparison
test_msg = "A" * 100

start = time.time()
rsa_enc = rsa_encrypt(test_msg, public_key)
rsa_dec = rsa_decrypt(rsa_enc, private_key)
rsa_time = time.time() - start

start = time.time()
aes_enc = aes_encrypt(test_msg, aes_key)
aes_dec = aes_decrypt(aes_enc, aes_key)
aes_time = time.time() - start

print(f"RSA time: {rsa_time * 1000:.2f}ms")
print(f"AES time: {aes_time * 1000:.2f}ms")
print(f"AES is ~{rsa_time / aes_time:.0f}x faster")

RSA time: 0.00ms
AES time: 2.99ms
AES is ~0x faster


### Comparison Table

| Feature | Symmetric (AES) | Asymmetric (RSA) |
|---------|-----------------|------------------|
| **Keys** | Single shared key | Public + Private pair |
| **Speed** | Very fast | Slow |
| **Key Size** | 128-256 bits | 2048-4096 bits |
| **Key Distribution** | Difficult | Easy |
| **Scalability** | O(n²) keys | O(n) keys |
| **Message Size** | Unlimited | Limited (~190 bytes) |
| **Use Case** | Bulk encryption | Key exchange, signatures |
| **Security Basis** | Substitution-permutation | Prime factorization |

In [7]:
# Key management scalability
def keys_needed(users):
    symmetric = users * (users - 1) // 2
    asymmetric = users * 2
    return symmetric, asymmetric


for n in [10, 100, 1000]:
    s, a = keys_needed(n)
    print(f"{n} users: Symmetric={s:,} keys, Asymmetric={a} keys")

10 users: Symmetric=45 keys, Asymmetric=20 keys
100 users: Symmetric=4,950 keys, Asymmetric=200 keys
1000 users: Symmetric=499,500 keys, Asymmetric=2000 keys


### Hybrid Encryption (Best Practice)

Real-world systems combine both:
1. **AES** encrypts data (fast)
2. **RSA** encrypts AES key (secure distribution)

In [8]:
def hybrid_encrypt(message, public_key):
    session_key = get_random_bytes(32)
    encrypted_data = aes_encrypt(message, session_key)
    encrypted_key = public_key.encrypt(
        session_key,
        padding.OAEP(mgf=padding.MGF1(hashes.SHA256()), algorithm=hashes.SHA256(), label=None),
    )
    return encrypted_key, encrypted_data


def hybrid_decrypt(encrypted_key, encrypted_data, private_key):
    session_key = private_key.decrypt(
        encrypted_key,
        padding.OAEP(mgf=padding.MGF1(hashes.SHA256()), algorithm=hashes.SHA256(), label=None),
    )
    return aes_decrypt(encrypted_data, session_key)


# Example
large_msg = "Secret data" * 100
enc_key, enc_data = hybrid_encrypt(large_msg, public_key)
decrypted = hybrid_decrypt(enc_key, enc_data, private_key)

print(f"Original length: {len(large_msg)}")
print(f"Encrypted key size: {len(enc_key)} bytes")
print(f"Encrypted data size: {len(enc_data)} bytes")
print(f"Match: {large_msg == decrypted}")

Original length: 1100
Encrypted key size: 256 bytes
Encrypted data size: 1120 bytes
Match: True


## Conclusion

**When to use:**
- **RSA:** Key exchange, digital signatures, small messages
- **AES:** Large files, databases, fast encryption
- **Hybrid:** Web (HTTPS), email (PGP), secure communications

**Key takeaways:**
- RSA solves key distribution but is slow
- AES is fast but requires secure key sharing
- Modern systems use both (hybrid encryption)
- Always use proper padding (OAEP) and key sizes (2048+ bits)